In [1]:
import os

In [2]:
import pandas as pd
from tool import YearMonth, yearmonth_range

data = pd.read_csv('data/ChronicleData_cov2-s.csv')

In [9]:
TrainRange = [YearMonth(2019, 12), YearMonth(2021, 6)]
TestRange = [YearMonth(2021, 7), YearMonth(2023, 1)]
print(int(TrainRange[1] - TrainRange[0]))
print(int(TestRange[1] - TestRange[0]))
sample_num = 200

18
18


In [4]:
def sample_YM(data, start_ym:YearMonth, end_ym:YearMonth, sample_num:int=1):
    sample_df = pd.DataFrame()
    sample_num_rs = []
    for ym in yearmonth_range(start_ym, end_ym):
        pool = data[(data['year']*12 + data['month']) == ym.to_month()]
        sample_num_r = min(sample_num, len(pool))
        sample_new = pool.sample(n=sample_num_r, replace=False).reset_index(drop=True)
        sample_df = pd.concat([sample_df, sample_new])
        sample_num_rs.append(sample_num_r)
    return sample_df, sample_num_rs

In [5]:
train, train_count = sample_YM(data, TrainRange[0], TrainRange[1], 200)
test, test_count = sample_YM(data, TestRange[0], TestRange[1], 200)

In [6]:
train_count

[1,
 35,
 38,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200]

In [7]:
test_count

[200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200]

In [10]:
train.to_csv(f'data/train_{sample_num}.csv', index=False)
test.to_csv(f'data/test_{sample_num}.csv', index=False)

In [11]:
# df to fasta

def df_to_fasta(data):
    fasta_list = [f">{row['header']}\n{row['sequence']}" for _, row in data.iterrows()]
    fasta_str = "\n".join(fasta_list)
    return fasta_str
def df_to_fasta_file(dataPath, fastaPath=None):
    data = pd.read_csv(dataPath)
    if fastaPath is None:
        fastaName = os.path.basename(dataPath).split('.')[0] + '.mfasta'
        fastaPath = os.path.join(os.path.dirname(dataPath), fastaName)
        
    fasta_str = df_to_fasta(data)
    
    with open(fastaPath, "w") as output_handle:
        output_handle.write(fasta_str)
    print(f"完成，已將DataFrame轉換為.mfasta格式並寫入{fastaPath}")
    
df_to_fasta_file(f'data/train_{sample_num}.csv')
df_to_fasta_file(f'data/test_{sample_num}.csv')

完成，已將DataFrame轉換為.mfasta格式並寫入data\train_200.mfasta
完成，已將DataFrame轉換為.mfasta格式並寫入data\test_200.mfasta


In [12]:
# fasta to feature
from work20230906 import ExtractFeatures
# 由於是調用遠在天邊的pse-in-one，所以要用絕對路徑
maindir = os.getcwd().replace('\\','/') + '/'
train_fasta_path = maindir + f'data/train_{sample_num}.mfasta'
test_fasta_path = maindir + f'data/test_{sample_num}.mfasta'
train_feature_path = maindir + f'data/train_feature_{sample_num}.csv'
test_feature_path = maindir + f'data/test_feature_{sample_num}.csv'

encode = ['pse.py','SC-PseAAC',['-lamada','5','-w','0.2']]

train_feature = ExtractFeatures(train_fasta_path, encode[0], encode[1], encode[2], train_feature_path)
test_feature = ExtractFeatures(test_fasta_path, encode[0], encode[1], encode[2], test_feature_path)


Calculating...
Done.
Used time: 6.72600007057s

Calculating...
Done.
Used time: 7.42499995232s



In [13]:
# add header to feature
def add_header_to_feature(data_feature, data):
    info_df = data[['header', 'sequence','year','month']]
    
    # 修改header(column name)，格式是{A: B}，A是原本的column name，B是新的column name
    # 這裡是把原本的column name改成f0, f1, f2, ..., fn
    # 而"header"保持不變
    
    new_columns = {i: f'f{i}' for i in range(len(data_feature.columns) - 1)}
    data_feature = data_feature.rename(columns=new_columns)
    data_feature = pd.concat([info_df, data_feature], axis=1)
    
    return data_feature
train_feature = pd.read_csv(train_feature_path,header=None)
test_feature = pd.read_csv(test_feature_path,header=None)
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train_feature_with_header = add_header_to_feature(train_feature, train)
test_feature_with_header = add_header_to_feature(test_feature, test)

train_feature_with_header.to_csv('data/train_feature_header.csv', index=False)
test_feature_with_header.to_csv('data/test_feature_header.csv', index=False)

# 統計

In [19]:
import numpy as np
import pandas as pd
from tool import YearMonth, yearmonth_range

In [20]:
def levenshtein_distance(seq1, seq2):
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    
    # 初始化一個 (len_seq1+1) x (len_seq2+1) 的矩陣
    dp = [[0 for _ in range(len_seq2 + 1)] for _ in range(len_seq1 + 1)]
    
    # 初始化第一列和第一行
    for i in range(len_seq1 + 1):
        dp[i][0] = i
    for j in range(len_seq2 + 1):
        dp[0][j] = j
    
    # 填充矩陣
    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            if seq1[i - 1] == seq2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j] + 1,      # 刪除
                               dp[i][j - 1] + 1,      # 插入
                               dp[i - 1][j - 1] + 1)  # 替換
    
    # Levenshtein距離
    return dp[len_seq1][len_seq2]

def calculate_ld_matrix(df):
    sequences = df['sequence'].tolist()
    n = len(sequences)
    ld_matrix = np.zeros((n, n), dtype=int)

    for i in range(n):
        for j in range(i+1, n):
            ld = levenshtein_distance(sequences[i], sequences[j])
            ld_matrix[i][j] = ld
            ld_matrix[j][i] = ld  # LD是對稱的

    return ld_matrix

def month_LDmatrix(data, start_ym:YearMonth, end_ym:YearMonth):
    ld_list = []
    for ym in yearmonth_range(start_ym, end_ym):
        pool = data[(data['year']*12 + data['month']) == ym.to_month()]
        ld_matrix = calculate_ld_matrix(pool)
        ld_list.append(ld_matrix)
        print(f"[ {ym.to_month()-start_ym.to_month()} / {end_ym.to_month()-start_ym.to_month()} ] {ym}的Levenshtein距離矩陣計算完成")
    return ld_list

In [21]:
TrainRange = [YearMonth(2019, 12), YearMonth(2021, 6)]
TestRange = [YearMonth(2021, 7), YearMonth(2023, 1)]
train_set = pd.read_csv('data/train_feature_header.csv')
test_set = pd.read_csv('data/test_feature_header.csv')

In [ ]:
# 保存Levenshtein距離矩陣的list
train_ld_list = month_LDmatrix(train_set, TrainRange[0], TrainRange[1])
np.savez_compressed('data/train_ld_list.npz', *train_ld_list)

In [82]:
test_ld_list = month_LDmatrix(test_set, TestRange[0], TestRange[1])
np.savez_compressed('data/test_ld_list.npz', *test_ld_list)

[ 0 / 18 ] 202107的Levenshtein距離矩陣計算完成
[ 1 / 18 ] 202108的Levenshtein距離矩陣計算完成
[ 2 / 18 ] 202109的Levenshtein距離矩陣計算完成
[ 3 / 18 ] 202110的Levenshtein距離矩陣計算完成
[ 4 / 18 ] 202111的Levenshtein距離矩陣計算完成
[ 5 / 18 ] 202112的Levenshtein距離矩陣計算完成
[ 6 / 18 ] 202201的Levenshtein距離矩陣計算完成
[ 7 / 18 ] 202202的Levenshtein距離矩陣計算完成
[ 8 / 18 ] 202203的Levenshtein距離矩陣計算完成
[ 9 / 18 ] 202204的Levenshtein距離矩陣計算完成
[ 10 / 18 ] 202205的Levenshtein距離矩陣計算完成
[ 11 / 18 ] 202206的Levenshtein距離矩陣計算完成
[ 12 / 18 ] 202207的Levenshtein距離矩陣計算完成
[ 13 / 18 ] 202208的Levenshtein距離矩陣計算完成
[ 14 / 18 ] 202209的Levenshtein距離矩陣計算完成
[ 15 / 18 ] 202210的Levenshtein距離矩陣計算完成
[ 16 / 18 ] 202211的Levenshtein距離矩陣計算完成
[ 17 / 18 ] 202212的Levenshtein距離矩陣計算完成
[ 18 / 18 ] 202301的Levenshtein距離矩陣計算完成


In [47]:
def load_ld_list(dataPath):
    with np.load(dataPath) as data:
        # 提取所有矩陣並返回為列表
        return [data[key] for key in data]

In [49]:
a = load_ld_list('data/train_ld_list.npz')

In [70]:
def stastic_ld(ld_matrix):
    mask = np.triu(np.ones(ld_matrix.shape), k=1).astype(bool)
    ld_matrix = ld_matrix[mask]
    ld_mean = np.mean(ld_matrix)
    ld_std = np.std(ld_matrix)
    ld_max = np.max(ld_matrix)
    ld_min = np.min(ld_matrix)
    return ld_mean, ld_std, ld_max, ld_min

In [81]:
for i, ld_matrix in enumerate(train_ld_list):
    if i == 0:
        continue
    ld_mean, ld_std, ld_max, ld_min = stastic_ld(ld_matrix)
    print(f"[ {i:0>2} ] 平均值: {ld_mean:.2f}, 標準差: {ld_std:.2f}, 最大值: {ld_max}, 最小值: {ld_min}")

[ 01 ] 平均值: 6.27, 標準差: 4.51, 最大值: 23, 最小值: 1
[ 02 ] 平均值: 6.30, 標準差: 8.24, 最大值: 46, 最小值: 1
[ 03 ] 平均值: 4.24, 標準差: 3.60, 最大值: 24, 最小值: 1
[ 04 ] 平均值: 4.58, 標準差: 3.62, 最大值: 23, 最小值: 1
[ 05 ] 平均值: 3.72, 標準差: 1.66, 最大值: 12, 最小值: 1
[ 06 ] 平均值: 3.80, 標準差: 1.61, 最大值: 13, 最小值: 1
[ 07 ] 平均值: 3.14, 標準差: 0.79, 最大值: 6, 最小值: 1
[ 08 ] 平均值: 6.01, 標準差: 4.85, 最大值: 34, 最小值: 2
[ 09 ] 平均值: 4.31, 標準差: 1.94, 最大值: 15, 最小值: 2
[ 10 ] 平均值: 4.72, 標準差: 2.86, 最大值: 23, 最小值: 2
[ 11 ] 平均值: 4.90, 標準差: 1.94, 最大值: 14, 最小值: 1
[ 12 ] 平均值: 4.51, 標準差: 1.53, 最大值: 11, 最小值: 2
[ 13 ] 平均值: 6.70, 標準差: 3.22, 最大值: 19, 最小值: 2
[ 14 ] 平均值: 8.77, 標準差: 3.79, 最大值: 21, 最小值: 2
[ 15 ] 平均值: 9.86, 標準差: 4.35, 最大值: 25, 最小值: 2
[ 16 ] 平均值: 13.75, 標準差: 5.75, 最大值: 25, 最小值: 2
[ 17 ] 平均值: 15.10, 標準差: 6.40, 最大值: 28, 最小值: 2
[ 18 ] 平均值: 16.07, 標準差: 7.77, 最大值: 28, 最小值: 2


In [85]:
for i, ld_matrix in enumerate(test_ld_list):
    ld_mean, ld_std, ld_max, ld_min = stastic_ld(ld_matrix)
    print(f"[ {i:0>2} ] 平均值: {ld_mean:.2f}, 標準差: {ld_std:.2f}, 最大值: {ld_max}, 最小值: {ld_min}")

[ 00 ] 平均值: 8.41, 標準差: 7.17, 最大值: 34, 最小值: 2
[ 01 ] 平均值: 6.53, 標準差: 4.97, 最大值: 26, 最小值: 2
[ 02 ] 平均值: 6.31, 標準差: 3.78, 最大值: 26, 最小值: 2
[ 03 ] 平均值: 5.80, 標準差: 2.02, 最大值: 15, 最小值: 2
[ 04 ] 平均值: 5.48, 標準差: 1.52, 最大值: 11, 最小值: 2
[ 05 ] 平均值: 20.32, 標準差: 18.35, 最大值: 49, 最小值: 2
[ 06 ] 平均值: 11.41, 標準差: 10.20, 最大值: 46, 最小值: 2
[ 07 ] 平均值: 12.58, 標準差: 11.36, 最大值: 43, 最小值: 2
[ 08 ] 平均值: 17.97, 標準差: 12.76, 最大值: 35, 最小值: 1
[ 09 ] 平均值: 11.07, 標準差: 11.45, 最大值: 41, 最小值: 2
[ 10 ] 平均值: 6.36, 標準差: 5.60, 最大值: 34, 最小值: 2
[ 11 ] 平均值: 7.09, 標準差: 2.87, 最大值: 15, 最小值: 1
[ 12 ] 平均值: 6.92, 標準差: 5.40, 最大值: 28, 最小值: 2
[ 13 ] 平均值: 5.57, 標準差: 2.86, 最大值: 17, 最小值: 1
[ 14 ] 平均值: 6.33, 標準差: 3.91, 最大值: 23, 最小值: 2
[ 15 ] 平均值: 8.84, 標準差: 6.93, 最大值: 36, 最小值: 2
[ 16 ] 平均值: 8.67, 標準差: 6.10, 最大值: 27, 最小值: 2
[ 17 ] 平均值: 9.43, 標準差: 5.80, 最大值: 22, 最小值: 1
[ 18 ] 平均值: 13.06, 標準差: 7.66, 最大值: 28, 最小值: 2
